Employee Attrition Project

In [0]:
%sql
select * from emp_attrition;

In [0]:
%sql
select sum(employeecount) from emp_attrition

In [0]:
%sql
select attrition, sum(EmployeeCount) from emp_attrition
group by attrition;

In [0]:
%sql
describe detail emp_attrition

In [0]:
%sql
select * from emp_attrition limit 10;

In [0]:
%sql
create table Customer_copy as (select * 
from samples.bakehouse.sales_customers limit 10)


In [0]:
%sql
select * from customer_copy

In [0]:
%sql
describe detail customer_copy

In [0]:
%sql
describe history default.customer_copy

In [0]:
%sql
Alter table customer_copy
ADD COLUMNS (Join_year INT);

In [0]:
%sql
update customer_copy
set Join_year = 2020;
    
select * from customer_copy

In [0]:
%sql
Update customer_copy
set Join_year = 2024
where first_name like 'A%';
    
select * from customer_copy

Time Travel

In [0]:
%sql
DESCRIBE HISTORY customer_copy

In [0]:
%sql
select * from customer_copy@v2;
select * from customer_copy version as of 3;

In [0]:
%sql
ALTER TABLE customer_copy 
SET TBLPROPERTIES (
    'delta.minReaderVersion' = '2', 
    'delta.minWriterVersion' = '5', 
    'delta.columnMapping.mode' = 'name'
);

ALTER TABLE customer_copy 
DROP COLUMN Join_year;

SELECT * FROM customer_copy;

Rollback to version

In [0]:
%sql
--using Restore command
Restore table default.customer_copy to version as of 4;

In [0]:
%sql
Optimize customer_copy
zorder by customerID;

In [0]:
%sql
select * from customer_copy;

In [0]:
%sql
describe history customer_copy;

In [0]:
%sql
vacuum customer_copy retain 0 hours;

In [0]:
%sql
Describe detail customer_copy;

CTAS with Delta tables

In [0]:
%sql
create table tab1
as select * from customer_copy;
    
select * from tab1;

In [0]:
%sql
drop table tab1

###Views

In [0]:
%sql
-- Temp View : temp view for a session, will be delted once session is restarted
create temp view cust_temp_view
as select * from customer_copy where last_name like 'M%';

In [0]:
%sql
select * from cust_temp_view

In [0]:
%sql
--- temp global view : Created to use from anywhere within cluster. Will be deleted if cluster restart.
--- can't be created on serverless compute
create global temp view cust_global_view
as select * from cust_temp_view;

select * from cust_global_view;

### Quering Files in Databricks

with 'USING' keyword:

In [0]:
%sql
create table Titanic_Tab
using csv
options (header = true, inferSchema = true, delimiter = ',')
location 'dbfs:/Workspace/Users/100ravsuman9155@gmail.com/Demo_Repo/My Data/titanic.csv'

Using JDBC connection:

In [0]:
%sql
create table Titanic_Tab
using jdbc
options(url = 'jdbc:mysql://localhost:3306/retail',
databse = 'retail',
driver = 'com.mysql.jdbc.Driver',
dbtable = 'titanic',
user = 'root',
password = 'root')

--> These are not delta tables, so we can't perform any delta operations or features on it
--> Like rollback, History etc
--> Solution for this is to create a temp view and then create Delta table from that view using CTAS

### Creating Delta Table

In [0]:
%sql
CREATE TEMP VIEW Titanic_View
USING csv
OPTIONS (
  header 'true',
  delimiter ',',
  inferSchema 'true',
  path '/My Data/titanic.csv'
);

CREATE TABLE Titanic_Tab
AS SELECT * FROM Titanic_View;

### Advanced Transformation
--- To extract/access column of JSON object type

In [0]:
%sql
select * from customer_copy

In [0]:
%sql
create or replace table cust_temp
as select * from customer_copy;

update cust_temp
set address = '{
	"street": "22 Rue du",
	"postalCode": "75003",
	"countryCode": "FRA"
}';

select * from cust_temp;



Extracting json object typle column value individually using ':'

In [0]:
%sql
select customerID, address:street, address:postalcode, address:countrycode from cust_temp;

By parsing JSON into Struct type using from_json() function:

---to use this function we need schema of the JSON_Object, can take one column value considered as schema

In [0]:
%sql
select address from cust_temp limit 1;
 

In [0]:
%sql

create or replace temp view cust_parsed_add
as select customerID, from_json(address, schema_of_json('{ "street": "22 Rue du", "postalCode": "75003", "countryCode": "FRA"}')/*, 'street string, postalCode int, countryCode string'*/) as address_struct
from cust_temp;

select * from cust_parsed_add;

--- Interacting with created struct datatype column using '.' operator and * operation:

In [0]:
%sql
---using . operator
select customerID, address_struct.street, address_struct.postalCode, address_struct.countryCode from cust_parsed_add;

In [0]:
%sql
--- using * operation
select customerId, address_struct.* from cust_parsed_add;

### Some Important function
1. Explode(): transform array or map type columns into multiple rows for that ID

In [0]:
%sql
create or replace temp view cust_array_item as 
select 
    customerID, 
    address, 
    array(
        struct('pen' as item, 20 as cost),
        struct('pencil' as item, 10 as cost),
        struct('eraser' as item, 5 as cost)
    ) as items 
from 
    customer_copy;

select * from cust_array_item;

In [0]:
%sql
select customerID, address, explode(items) as item
from cust_array_Item;

In [0]:
%sql
create temp view cust_item_view
as select customerID, address, explode(items) as item
from cust_array_Item;

(2) Collect_set(): used to collect values of records into an array for same ID(columns) 

In [0]:
%sql
select customerID, collect_set(item) as items
from cust_item_view
group by customerID;

3) Flatten() and array_distinct(): flatten used to flatten the array list created after collect_set, and array_distinct used to remove duplicates value from flattened values.
-- works with array of array datatype

In [0]:
%sql
--
select customerID, array_distinct(flatten(collect_set(item))) as items_list
from cust_item_view
group by customerID;

### Higher Order functions
---> Allows us to work with Hierarichal data like Array and Map type objects 
1) Filter(): filters for any key of map or array

In [0]:
%sql
select customerID, items,
  filter(items, i -> i.cost > 10) as costly_pen
from cust_array_item

2) Transform() : used to transform the values of keys in an array and extract them

In [0]:
%sql
select customerID, items,
  filter(items, i-> i.item = 'pen') as isItPen
from cust_array_item

### User defined Functions (UDFs):
--> defined by user with passing arguments with datatype and returning values after process with datatype

In [0]:
%sql
create or replace function site_type(email String) returns string
return case
   when email like '%.com' then 'Commercial Website'
   when email like '%.org' then 'Organizational Website'
   when email like '%.edu' then 'educational website'
   else 'unknown'
end;

select customerID, email_address, site_type(email_address) as Extension
from customer_copy;
